Prompt(Chatgpt 4o): How to calculate FID and Clip score of trained emoji generator model

In [ ]:
# Imports
import torch
from diffusers import StableDiffusionPipeline
from peft import PeftModel
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel

In [ ]:


# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Prompts (your 30)
PROMPTS = [
    "happy face with sunglasses", "crying emoji", "smiling ghost", "angry robot",
    "winking face", "sleeping moon emoji", "party popper explosion", "emoji with heart eyes",
    "thinking face emoji", "face with mask", "fire emoji", "thumbs up", "clapping hands",
    "shocked face", "money eyes emoji", "face vomiting", "zany face emoji", "face with monocle",
    "pleading eyes emoji", "face screaming in fear", "nerd face", "cold face", "melting face",
    "face with spiral eyes", "face with symbols on mouth", "hugging face", "smiling face with halo",
    "face with raised eyebrow", "emoji blowing a kiss", "saluting face"
]

# Output directory
save_dir = "outputs/manual_prompt_clip_only"
os.makedirs(save_dir, exist_ok=True)

# Load Stable Diffusion with LoRA
model_id = "sd-legacy/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(device)
pipe.unet = PeftModel.from_pretrained(pipe.unet, "../evaluation/lora_google_emoji_diffusion/final_model")
pipe.enable_attention_slicing()
pipe.safety_checker = lambda images, clip_input, **kwargs: (images, [False] * len(images))
print("✅ LoRA model loaded and ready.")

# Generate images
results = []
for i, prompt in enumerate(tqdm(PROMPTS, desc="Generating images")):
    result = pipe(prompt, num_inference_steps=50, guidance_scale=7.5, height=256, width=256)
    image = result.images[0]
    image_path = os.path.join(save_dir, f"emoji_{i:02}.png")
    image.save(image_path)
    results.append({"prompt": prompt, "generated_image_path": image_path})

results_df = pd.DataFrame(results)

# Load CLIP model
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").eval().to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Compute CLIP scores
clip_scores = []
for _, row in tqdm(results_df.iterrows(), total=len(results_df), desc="Calculating CLIP scores"):
    image = Image.open(row["generated_image_path"]).convert("RGB")
    inputs = clip_processor(text=row["prompt"], images=image, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        # Run processor
        inputs = clip_processor(text=row["prompt"], images=image, return_tensors="pt", padding=True).to(device)

        # Extract pixel and text features separately
        image_inputs = {"pixel_values": inputs["pixel_values"]}
        text_inputs = {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"]
        }

        # Get features
        image_embeds = clip_model.get_image_features(**image_inputs)
        text_embeds = clip_model.get_text_features(**text_inputs)

        # Normalize and compute cosine similarity
        image_embeds = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
        text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)

        score = torch.nn.functional.cosine_similarity(image_embeds, text_embeds).item()
        clip_scores.append(score)

results_df["clip_score"] = clip_scores
results_df.to_parquet(os.path.join(save_dir, "clip_scores.parquet"), index=False)

# Print summary
print(f"\n All done! Results saved to: {save_dir}")
print(f" Average CLIP Score: {results_df['clip_score'].mean():.4f}")

Using device: cuda


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

✅ LoRA model loaded and ready.


Generating images:   0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Generating images:  97%|█████████▋| 29/30 [20:58<00:43, 43.19s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

Calculating CLIP scores: 100%|██████████| 30/30 [00:01<00:00, 28.12it/s]


 All done! Results saved to: outputs/manual_prompt_clip_only
 Average CLIP Score: 0.2712


In [1]:
import torch
import os
import pandas as pd
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets.folder import default_loader
from tqdm import tqdm
from diffusers import StableDiffusionPipeline
from peft import PeftModel
from PIL import Image
from pytorch_fid.fid_score import calculate_fid_given_paths

# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load validation dataset
df = pd.read_parquet("../data/processed_emoji_dataset.parquet")

class EmojiDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_tensor = torch.load(self.df.iloc[idx]['image_path']).float()
        prompt = self.df.iloc[idx]['prompt']
        return {"pixel_values": image_tensor, "prompt": prompt}

dataset = EmojiDataset(df)
train_size = int(0.97 * len(dataset))
_, val_set = random_split(dataset, [train_size, len(dataset) - train_size])

# Generate images from validation prompts
model_id = "sd-legacy/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(device)
pipe.unet = PeftModel.from_pretrained(pipe.unet, "../evaluation/lora_google_emoji_diffusion/final_model")
pipe.enable_attention_slicing()
pipe.safety_checker = lambda images, clip_input, **kwargs: (images, [False] * len(images))
print("✅ LoRA model loaded.")

# Output directories for FID
generated_dir = "outputs/fid/generated"
real_dir = "outputs/fid/real"
os.makedirs(generated_dir, exist_ok=True)
os.makedirs(real_dir, exist_ok=True)

print("🖼 Generating validation images...")
for i, sample in enumerate(tqdm(val_set)):
    prompt = sample["prompt"]
    real_image_tensor = sample["pixel_values"]
    
    # Save real image
    real_image = transforms.ToPILImage()(real_image_tensor)
    real_image.save(os.path.join(real_dir, f"real_{i:04}.png"))

    # Generate and save fake image
    result = pipe(prompt, num_inference_steps=50, guidance_scale=7.5, height=256, width=256)
    gen_image = result.images[0]
    gen_image.save(os.path.join(generated_dir, f"gen_{i:04}.png"))

# Compute FID using pytorch_fid
print("📊 Calculating FID...")
fid_score = calculate_fid_given_paths([real_dir, generated_dir], batch_size=32, device=device, dims=2048)
print(f"✅ FID Score: {fid_score:.4f}")


Using device: cuda


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

✅ LoRA model loaded.
🖼 Generating validation images...


  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 91/91 [06:51<00:00,  4.52s/it]
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to C:\Users\Jyoti Prakash Uprety/.cache\torch\hub\checkpoints\pt_inception-2015-12-05-6726825d.pth


📊 Calculating FID...


100%|██████████| 91.2M/91.2M [00:01<00:00, 67.4MB/s]
100%|██████████| 3/3 [00:04<00:00,  1.39s/it]


✅ FID Score: 174.8789
